# Agent Build with Langchain Agents and watsonx.data document libraray retrival MCP Server

#### Install the dependencies

In [ ]:
!pip install ibm-watsonxdata-dl-retrieval-mcp-server langchain-mcp-adapters langgraph langchain ibm_watson_machine_learning langchain_ibm

#### Add the environment variables

In [ ]:

import os

os.environ["WATSONX_API_KEY"]="<>"
os.environ["WATSONX_PROJECT_ID"]="<>"
os.environ["WATSONX_MODEL_ID"] = "meta-llama/llama-3-3-70b-instruct"
os.environ["WATSONX_URL"]="https://eu-de.ml.cloud.ibm.com"

#### Imports

In [50]:

from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import ChatWatsonx
import os
import asyncio

#### Setting up ChatWatsonx llm

In [51]:
parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 8000,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.7,
    }

model = ChatWatsonx(
    model_id=os.getenv("WATSONX_MODEL", "meta-llama/llama-3-3-70b-instruct"),
    apikey=os.getenv("WATSONX_API_KEY"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    url=os.getenv("WATSONX_URL", "https://eu-de.ml.cloud.ibm.com"),
    parameters=parameters
)


### MCP Client creation (SSE Transport mode) 

In [52]:
client = MultiServerMCPClient(
    {
        "dl_retrieval": {
            "transport": "sse",
            "url": "http://localhost:8000/sse",
            "init_delay": 5.0
        },
    })

print("Created the mcp client in sse transport mode")

Created the mcp client in sse transport mode


#### List the available tools

In [ ]:

async def printAvailableTools():  
    tools = []
    try:
        print("Attempting to create session...")
        async with client.session(server_name="dl_retrieval") as session:
            try:
                print("session created")
                mcp_tools = await load_mcp_tools(session)
                tools.extend(mcp_tools)

                print("\nAvailable Tools:")
                for i, tool in enumerate(tools, 1):
                    print(f"{i}. {tool.name}")
                    print(f"   Description: {tool.description}")
                    print("-" * 50)
            except Exception as e:
                    print(f"Failed to load MCP tools: {e}")
                    raise
    except Exception as e:
        print(f"Failed to create session: {e}")
        raise

await printAvailableTools()

#### Function for creating and invoking the langchain agent

In [ ]:
async def createLangchainAgentAndRun():   
    try:
        tools = []
        print("Attempting to create session...")
        async with client.session(server_name="dl_retrieval") as session:
                print("session created")
                mcp_tools = await load_mcp_tools(session)
                tools.extend(mcp_tools)
                dl_agent = create_react_agent(model=model, tools=tools,prompt=(
                    "You are document library search and response agent"
                    "Your task is to answer the user query by calling the relevant tools given to you"
                    "Always show the output of the function call"
                ))
                                                                            
                user_input = "What is the total invoice count"
                
                print("Agent created. Ready to interact!")
                
                # invocation
                try:
                
                    result = await dl_agent.ainvoke({
                        "messages": [
                            {
                                "role": "user",
                                "content": user_input
                            }
                        ]
                    })
                    print(result)
                except Exception as e:
                    print(f"Error: {str(e)}")
                    raise
    except Exception as e:
        print(f"Failed to create session: {e}")
        raise

await createLangchainAgentAndRun()
        
                